In [1]:
import os

In [2]:
data_file='quora_duplicate_questions.tsv'
# 0 means dont load, 1 means fetch from file
LOAD_ENCODING_FROM_FILE=0 
encoding_data_file_quest1='encoding_quest1'
encoding_data_file_quest2='encoding_quest2'
encoding_data_file_label='label'

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from bert_serving.client import BertClient
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

/home/swayam/anaconda3/envs/dia/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def sanitize_question(text):
    ''' Pre process and convert texts to a list of words 
    '''
    text = str(text)
    text = text.lower()
# Clean the text
    text = re.sub(r"[^A-Za-z0–9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #text = text.split()
    return text

In [5]:
# Read the input file
data_df=pd.read_csv(data_file,sep='\t',nrows=1000)
print(data_df.shape)

(1000, 6)


In [6]:
data_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
sent1=[]
sent2=[]
label=[]
for index, row in data_df.iterrows():
	sent1.append(sanitize_question(row[3]))
	sent2.append(sanitize_question(row[4]))
	label.append(sanitize_question(row[5]))

In [8]:
maxlen = 256

In [9]:
if LOAD_ENCODING_FROM_FILE == 0:
    bc=BertClient(port=5555, port_out=5556)
    vec11 = []
    for i in sent1:
        vec1=bc.encode([i])
        vec11.append(vec1)
    with open(encoding_data_file_quest1, "wb") as fp:
        pickle.dump(vec11, fp)
        
    vec22 = []
    for ii in sent2:
        vec2=bc.encode([i])
        vec22.append(vec2)
    with open(encoding_data_file_quest2, "wb") as fp:   
        pickle.dump(vec22,fp)
      
    with open(encoding_data_file_label, "wb") as fp: 
        pickle.dump(label,fp)
exit(0)